# 📚 Translation Benchmark Notebook
**Test translation speed & performance across models, parameters, and languages**

Upload books, configure models, translate, and download results.
Helps you decide if Colab Pro+ is worth it!

In [ ]:
#@title 1️⃣ Setup & Installation
#@markdown Run this cell first

!pip install -q transformers torch accelerate sentencepiece protobuf PyPDF2 ebooklib beautifulsoup4

from google.colab import files as colab_files
import os, time, json, re
from datetime import datetime
from pathlib import Path

OUTPUT_DIR = '/content/translations'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('✅ Setup complete!')

BENCHMARK_LOG = []

In [ ]:
#@title 2️⃣ Model Provider Selection
#@markdown Choose provider (local models are FREE)

import torch

provider = 'huggingface_local' #@param ['huggingface_local', 'groq_free', 'google_free']

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'🎮 GPU: {gpu_name} ({gpu_mem:.1f} GB)')
else:
    print('⚠️ No GPU - using CPU')

api_key = '' #@param {type:'string'}
if provider in ['groq_free', 'google_free'] and not api_key:
    print('⚠️ API key needed for cloud providers')

In [ ]:
#@title 3️⃣ Model Configuration

model_name = 'Helsinki-NLP/opus-mt-en-hi' #@param {type:'string'}
#@markdown Suggested: `Helsinki-NLP/opus-mt-en-hi`, `facebook/nllb-200-distilled-600M`, `google/flan-t5-base`

temperature = 0.3 #@param {type:'slider', min:0, max:1, step:0.1}
top_p = 0.9 #@param {type:'slider', min:0, max:1, step:0.1}
max_tokens = 4096 #@param {type:'slider', min:512, max:8192, step:512}

print(f'📦 Model: {model_name}')

In [ ]:
#@title 4️⃣ Translation Settings

source_language = 'English' #@param {type:'string'}
target_language = 'Hindi' #@param {type:'string'}
#@markdown Enter any language your model supports

tier = 'BASIC' #@param ['BASIC', 'INTERMEDIATE', 'ADVANCED']
chunk_words = 350 #@param {type:'slider', min:100, max:600, step:50}

print(f'🌍 {source_language} → {target_language}')

In [ ]:
#@title 5️⃣ Chunk Overlapping
#@markdown Enable for better context between chunks

enable_overlap = True #@param {type:'boolean'}
overlap_words = 50 #@param {type:'slider', min:10, max:150, step:10}

if enable_overlap:
    print(f'✅ Overlap: {overlap_words} words')
else:
    print('⚠️ Overlapping disabled')

In [ ]:
#@title 6️⃣ Upload Book

import PyPDF2
from io import BytesIO

print('📤 Upload your book...')
uploaded = colab_files.upload()

book_name = list(uploaded.keys())[0]
book_content = uploaded[book_name]

if book_name.endswith('.pdf'):
    pdf = PyPDF2.PdfReader(BytesIO(book_content))
    text = '\n'.join([p.extract_text() or '' for p in pdf.pages])
elif book_name.endswith('.epub'):
    import ebooklib
    from ebooklib import epub
    from bs4 import BeautifulSoup
    book = epub.read_epub(BytesIO(book_content))
    text = ''.join([BeautifulSoup(i.get_content(), 'html.parser').get_text() for i in book.get_items_of_type(ebooklib.ITEM_DOCUMENT)])
else:
    text = book_content.decode('utf-8')

word_count = len(text.split())
print(f'📖 {book_name}: {word_count:,} words')

In [ ]:
#@title 7️⃣ Load Model

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
import requests

translator = None

if provider == 'huggingface_local':
    print(f'📥 Loading {model_name}...')
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if 't5' in model_name.lower() or 'mt' in model_name.lower() or 'nllb' in model_name.lower():
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
            translator = pipeline('translation', model=model, tokenizer=tokenizer, device_map='auto')
        else:
            model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
            translator = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto', max_new_tokens=max_tokens)
        print('✅ Model loaded!')
    except Exception as e:
        print(f'❌ Error: {e}')
else:
    print('✅ API ready')

In [ ]:
#@title 8️⃣ Run Translation

import warnings
warnings.filterwarnings('ignore')

def chunk_text(text, chunk_words, overlap_words, enable_overlap):
    words = text.split()
    chunks = []
    step = chunk_words - overlap_words if enable_overlap else chunk_words
    i = 0
    while i < len(words):
        chunks.append(' '.join(words[i:i+chunk_words]))
        i += step
    return chunks

def api_translate(chunk, provider, model_name, api_key, target_lang):
    if provider == 'groq_free':
        r = requests.post('https://api.groq.com/openai/v1/chat/completions',
            headers={'Authorization': f'Bearer {api_key}'},
            json={'model': model_name, 'messages': [{'role': 'user', 'content': f'Translate to {target_lang}: {chunk}'}]})
        return r.json()['choices'][0]['message']['content']
    elif provider == 'google_free':
        r = requests.post(f'https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent?key={api_key}',
            json={'contents': [{'parts': [{'text': f'Translate to {target_lang}: {chunk}'}]}]})
        return r.json()['candidates'][0]['content']['parts'][0]['text']
    return chunk

chunks = chunk_text(text, chunk_words, overlap_words, enable_overlap)
print(f'📦 {len(chunks)} chunks')

translations = []
chunk_times = []
start_time = time.time()

for i, chunk in enumerate(chunks, 1):
    t0 = time.time()
    print(f'\r⏳ {i}/{len(chunks)}', end='')
    try:
        if provider == 'huggingface_local' and translator:
            if 't5' in model_name.lower() or 'mt' in model_name.lower() or 'nllb' in model_name.lower():
                result = translator(chunk, max_length=max_tokens)[0]['translation_text']
            else:
                result = translator(f'Translate to {target_language}: {chunk}')[0]['generated_text']
        else:
            result = api_translate(chunk, provider, model_name, api_key, target_language)
        translations.append(result)
    except Exception as e:
        translations.append(f'[ERROR: {e}]')
    chunk_times.append(time.time() - t0)

total_time = time.time() - start_time
print(f'\n✅ Done in {total_time/60:.1f} min')

In [ ]:
#@title 9️⃣ Performance & Colab Pro+ Analysis

avg_time = sum(chunk_times) / len(chunk_times)
chars_sec = len(text) / total_time

gpu_mem = torch.cuda.max_memory_allocated() / 1e9 if torch.cuda.is_available() else 0

print('='*50)
print('📊 METRICS')
print('='*50)
print(f'⏱️ Total: {total_time/60:.1f} min')
print(f'📦 Avg chunk: {avg_time:.1f}s')
print(f'⚡ Speed: {chars_sec:.0f} chars/sec')
print(f'🎮 GPU: {gpu_mem:.1f} GB')

print('\n' + '='*50)
print('💎 COLAB PRO+ COMPARISON')
print('='*50)

gpu_data = {
    'L4 GPU': {'x': 2.5, 'vram': '24GB', 'models': 'Up to 13B'},
    'A100 GPU': {'x': 5, 'vram': '40GB', 'models': 'Up to 30B'},
    'H100 GPU': {'x': 8, 'vram': '80GB', 'models': 'Up to 70B'},
    'v6e TPU': {'x': 4, 'vram': '16GB HBM', 'models': 'Transformers optimized'}
}

for gpu, d in gpu_data.items():
    print(f"\n{gpu}: {total_time/d['x']/60:.1f} min ({d['x']}x faster)")
    print(f"  💾 {d['vram']} | 🤖 {d['models']}")

print('\n💡 ', end='')
if total_time > 600:
    print('Pro+ recommended for time savings')
elif gpu_mem > 12:
    print('Pro+ recommended for larger models')
else:
    print('Free tier works for this workload')

In [ ]:
#@title 🔟 Download or Copy Translation

from IPython.display import display, HTML

# Generate filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
book_base = Path(book_name).stem
model_short = model_name.split('/')[-1]
filename = f'{book_base}_{model_short}_{target_language}_{tier}_{timestamp}'

full_translation = '\n\n'.join(translations)

# Save locally
output_path = f'{OUTPUT_DIR}/{filename}.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(full_translation)

# Track benchmark
BENCHMARK_LOG.append({'book': book_name, 'model': model_name, 'target_lang': target_language,
                      'tier': tier, 'total_time_sec': total_time, 'chars_per_sec': chars_sec})

print('='*50)
print('📥 DOWNLOAD OPTIONS')
print('='*50)
print(f'📄 {filename}.txt ({len(full_translation):,} chars)\n')

# Option 1: Download
print('🔽 Option 1: Download file')
colab_files.download(output_path)

# Option 2: Copy button
escaped = full_translation.replace('\\', '\\\\').replace('`', '\\`').replace('"', '\\"').replace('\n', '\\n')
html = f'''<div style="margin:15px 0">
<button onclick="navigator.clipboard.writeText(`{escaped}`).then(()=>{{this.innerHTML='✅ Copied!';setTimeout(()=>this.innerHTML='📋 Copy All Text',2000)}})" 
style="padding:12px 24px;background:linear-gradient(135deg,#667eea,#764ba2);color:white;border:none;border-radius:8px;cursor:pointer;font-size:14px;font-weight:bold">
📋 Copy All Text</button></div>'''
print('\n📋 Option 2: Copy to clipboard')
display(HTML(html))

# Preview
print('\n👀 Preview (first 500 chars):\n')
print(full_translation[:500] + '...' if len(full_translation) > 500 else full_translation)

In [ ]:
#@title 📊 Comparison Dashboard

import pandas as pd

if BENCHMARK_LOG:
    df = pd.DataFrame(BENCHMARK_LOG)
    df['time_min'] = df['total_time_sec'] / 60
    display(df[['book', 'model', 'target_lang', 'tier', 'time_min', 'chars_per_sec']].style.background_gradient(cmap='RdYlGn_r'))
else:
    print('No runs yet')